# Using `bmfm-multi-omics` for zero-shot cell type annotation

Assigning cell type annotations is an important and time consuming part of single-cell analysis using `biomed-multi-omic` for cell type annotation. BMFM-RNA simplifies this process by not only performing the cell-type annotation but also the preprocessing and visualisation through the embeddings created by the model.

In [1]:
from pathlib import Path

import scanpy as sc

## Load Example Data

To demostrate the BMFM-RNAs abilites, we use the PBMC data created by 10X Genomics (dataset can be downloaded [here](https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/pbmc3k)). This dataset is created of 3k PBMCs from a Healthy Donor. The raw data will be used as the input, but we will also extract the cell type annotation from the legacy scanpy workflow as a comparison between the BMFM and classical scRNA-seq analysis. 

For more information about how the data was preprocessed please visit scanpy's tutorial [here](https://scanpy.readthedocs.io/en/1.11.x/tutorials/basics/clustering-2017.html).

In [2]:
# Get raw PBMC3k data
data_dir = Path("data")
data_dir.mkdir(parents=True, exist_ok=True)

# Get PMBC3k raw dataset
adata = sc.datasets.pbmc3k()

# Extract reference data for later downstream comparison
reference_adata = sc.datasets.pbmc3k_processed()
reference_obs_index = reference_adata.obs.index.tolist()
reference_vars_index = reference_adata.var.index.tolist()

adata = adata[reference_obs_index, reference_vars_index]
adata.write("data/pbmc3k_raw.h5ad")

In [3]:
# Make results directory
results_dir = Path("results/pbmc3k")
results_dir.mkdir(parents=True, exist_ok=True)

The model's weights can be aquired from [IBM's HuggingFace collection](https://huggingface.co/ibm-research). The following scRNA models are avaliable:

- MLM+RDA: [ibm-research/biomed.rna.bert.110m.mlm.rda.v1](https://huggingface.co/ibm-research/biomed.rna.bert.110m.mlm.rda.v1)
- MLM+Multitask: [ibm-research/biomed.rna.bert.110m.mlm.multitask.v1](https://huggingface.co/ibm-research/biomed.rna.bert.110m.mlm.multitask.v1)
- WCED+Multitask: [ibm-research/biomed.rna.bert.110m.wced.multitask.v1](https://huggingface.co/ibm-research/biomed.rna.bert.110m.wced.multitask.v1)
- WCED 10 pct: [ibm-research/biomed.rna.bert.110m.wced.v1](https://huggingface.co/ibm-research/biomed.rna.bert.110m.wced.v1)

Using `bmfm-targets-run` you will only need to provide the name of the model under the `checkpoint` flag. I.e. `checkpoint=ibm-research/biomed.rna.bert.110m.wced.multitask.v1`. Checkpoints will be downloaded automatically from HuggingFace.

To get embeddings for an h5ad file from the checkpoints discussed in the manuscript (https://arxiv.org/abs/2506.14861) run the following code snippets, after installing the package.

The only thing you need is an `h5ad` file with raw gene counts to run inference, and a writable directory working_dir for output. For convenience, this tutorial uses pmbc3k dataset created in the code chunks above, however, you could also provide your own `h5ad` file (note for WCED the expected input of the data in `.X` should be raw counts).

In [ ]:
%%bash
bmfm-targets-run -cn predict input_file=data/subset_hvg.h5ad working_dir=results/hvg checkpoint=ibm-research/biomed.rna.bert.110m.wced.multitask.v1